# Checking Embeddings of Terms (Noun/Verb/Adj/etc.) from Tagged Wordnet Gloss

I discovered there's a more active fork of wordnet and bumped this analysis over to that.

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("jon-tow/open-english-wordnet-synset-2023")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset['train'][0]

{'@id': 'oewn-03159292-a',
 '@ili': 'i18097',
 '@members': 'oewn-avenged-a',
 '@partOfSpeech': 'a',
 '@lexfile': 'adj.ppl',
 'Definition': 'for which vengeance has been taken',
 'Example': 'an avenged injury',
 'SynsetRelation': [],
 'ILIDefinition': None,
 '@dc:source': None}

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

,@id,@ili,@members,@partOfSpeech,@lexfile,Definition,Example,SynsetRelation,ILIDefinition,@dc:source
0,oewn-03159292-a,i18097,oewn-avenged-a,a,adj.ppl,for which vengeance has been taken,an avenged injury,[],None,None
1,oewn-03159419-a,i18098,oewn-unavenged-a,a,adj.ppl,for which vengeance has not been taken,an unavenged murder,[],None,None
2,oewn-03159554-a,i18099,oewn-beaten-a,a,adj.ppl,formed or made thin by hammering,beaten gold,[],None,None
3,oewn-03159654-a,i18100,oewn-calibrated-a oewn-graduated-a,a,adj.ppl,marked with or divided into degrees,a calibrated thermometer,[],None,None
4,oewn-03159804-a,i18101,oewn-cantering-a,a,adj.ppl,riding at a gait between a trot and a gallop,the cantering soldiers,[],None,None


Getting the @members into a reasonable format is about to take a bunch of cells and most of my patience for the day.

In [ ]:
df.shape

(120135, 10)

In [ ]:
df[['@members', '@partOfSpeech', '@lexfile']].head()

,@members,@partOfSpeech,@lexfile
0,oewn-avenged-a,a,adj.ppl
1,oewn-unavenged-a,a,adj.ppl
2,oewn-beaten-a,a,adj.ppl
3,oewn-calibrated-a oewn-graduated-a,a,adj.ppl
4,oewn-cantering-a,a,adj.ppl


In [ ]:
df = df[['@members', '@partOfSpeech', '@lexfile']]

In [ ]:
pattern = r'^(\w+-\w+-\w+ *)*$'

matches_pattern = df['@members'].str.match(pattern)

all_match_pattern = matches_pattern.all()
all_match_pattern

False

In [ ]:
members_not_matching_pattern = df[~matches_pattern]
members_not_matching_pattern

,@members,@partOfSpeech,@lexfile
13,oewn-hand-held-a oewn-handheld-a,a,adj.ppl
42,oewn-re-created-a,a,adj.ppl
49,oewn-spray-dried-a,a,adj.ppl
57,oewn-closed-captioned-a,a,adj.ppl
116,oewn-plane_figure-n oewn-two-dimensional_figure-n,n,noun.shape
...,...,...,...
119954,oewn-real-time_processing-n oewn-real-time_ope...,n,noun.process
119976,oewn-reuptake-n oewn-re-uptake-n,n,noun.process
120005,oewn-slump-n oewn-slack-n oewn-drop-off-n oewn...,n,noun.process
120123,oewn-constant-volume_process-n oewn-isometric_...,n,noun.process


In [ ]:
pattern = r'^(\w+-[\w_\-.]+-\w+ *)*$' # This took a couple of iterations not represented

matches_pattern = df['@members'].str.match(pattern)

all_match_pattern = matches_pattern.all()
all_match_pattern

True

In [ ]:
# I found this problem later on
oddball = df['@members'].str.match('.*Gravenhage.*')
oddball_member = df[oddball]['@members']

In [ ]:
oddball_member.iloc[0]

'oewn-The_Hague-n oewn--ap-s_Gravenhage-n oewn-Den_Haag-n'

In [ ]:
df = df.assign(members=df['@members'].str.split()).explode('members')

In [ ]:
df.head()

,@members,@partOfSpeech,@lexfile,members
0,oewn-avenged-a,a,adj.ppl,oewn-avenged-a
1,oewn-unavenged-a,a,adj.ppl,oewn-unavenged-a
2,oewn-beaten-a,a,adj.ppl,oewn-beaten-a
3,oewn-calibrated-a oewn-graduated-a,a,adj.ppl,oewn-calibrated-a
3,oewn-calibrated-a oewn-graduated-a,a,adj.ppl,oewn-graduated-a


In [ ]:
df.shape

(212071, 4)

In [ ]:
prefixes = df['members'].str.split('-', expand=True)[0]
prefix_freq = prefixes.value_counts().reset_index()
prefix_freq.columns = ['Prefix', 'Frequency']

prefix_freq = prefix_freq.sort_values(by='Frequency', ascending=False)

print(prefix_freq)

  Prefix  Frequency
0   oewn     212071


In [ ]:
to_remove = 'oewn-'

df['members'] = df['members'].apply(lambda x: x.replace(to_remove, ''))

In [ ]:
prefixes = df['members'].str.split('-', expand=True)[0]
prefix_freq = prefixes.value_counts().reset_index()
prefix_freq.columns = ['Prefix', 'Frequency']

prefix_freq = prefix_freq.sort_values(by='Frequency', ascending=False)

print(prefix_freq)

                                                Prefix  Frequency
0                                                 self        252
1                                                 high        102
2                                                 well         98
3                                                  one         85
4                                                  cut         79
...                                                ...        ...
66494                                              CIA          1
66493   National_Institute_of_Standards_and_Technology          1
66492                          Counterterrorist_Center          1
66491                          Nonproliferation_Center          1
145809                            grammatical_cohesion          1

[145810 rows x 2 columns]


In [ ]:
# Check for values starting with "-"
values_starting_with_dash = df[df['members'].str.startswith('-')]

# Display the values starting with "-"
print(values_starting_with_dash)

                                                 @members @partOfSpeech  \
81633   oewn-The_Hague-n oewn--ap-s_Gravenhage-n oewn-...             n   
106115                    oewn-between-r oewn--ap-tween-r             r   
107858        oewn-between_decks-r oewn--ap-tween_decks-r             r   
114349                        oewn-hood-n oewn--ap-hood-n             n   

             @lexfile             members  
81633   noun.location  -ap-s_Gravenhage-n  
106115        adv.all         -ap-tween-r  
107858        adv.all   -ap-tween_decks-r  
114349     noun.group          -ap-hood-n  


In [ ]:
df.head()

,@members,@partOfSpeech,@lexfile,members
0,oewn-avenged-a,a,adj.ppl,avenged-a
1,oewn-unavenged-a,a,adj.ppl,unavenged-a
2,oewn-beaten-a,a,adj.ppl,beaten-a
3,oewn-calibrated-a oewn-graduated-a,a,adj.ppl,calibrated-a
3,oewn-calibrated-a oewn-graduated-a,a,adj.ppl,graduated-a


In [ ]:
df['members'] = df['members'].apply(lambda x: x[4:] if x.startswith('-ap-') else x)

In [ ]:
df.drop(columns=['@members'], inplace=True)

In [ ]:
suffixes = df['members'].str.split('-').str[-1]

# Count frequencies of suffixes
suffix_freq = suffixes.value_counts().reset_index()
suffix_freq.columns = ['Suffix', 'Frequency']

# Sort by frequency
suffix_freq = suffix_freq.sort_values(by='Frequency', ascending=False)

# Display suffixes ordered by frequency
print(suffix_freq[:40])

  Suffix  Frequency
0      n     151001
1      a      30150
2      v      25098
3      r       5595
4      1        146
5      2         69
6      s         12


In [ ]:
filtered_df = df[df['members'].str.endswith(('1', '2', 's'))]

print(filtered_df)

       @partOfSpeech            @lexfile      members
286                n          noun.shape     lead-n-1
301                n          noun.shape      bow-n-1
325                n          noun.shape    tower-n-1
782                s             adj.all   panelled-s
2303               s             adj.all     centre-s
...              ...                 ...          ...
117472             v           verb.body     tear-v-2
117596             v           verb.body  recover-v-1
118299             v  verb.communication      bow-v-1
118397             v  verb.communication      bow-v-1
118473             v  verb.communication    whoop-v-1

[227 rows x 3 columns]


In [ ]:
df['members'] = df['members'].apply(lambda x: x.replace('-ap-', "'")) # They use this for apostrophe for some reason, probably because it was stored as yaml

In [ ]:
# List of suffixes to remove
suffixes_to_remove = ['-n', '-a', '-v', '-r', '-1', '-2', '-s']

# Function to remove suffixes
def remove_suffixes(member):
    # Iterate until no suffixes are left
    while any(member.endswith(suffix) for suffix in suffixes_to_remove):
        for suffix in suffixes_to_remove:
            if member.endswith(suffix):
                member = member[:-len(suffix)]  # Remove the suffix
    return member

# Apply the function to each member in the DataFrame
df['members'] = df['members'].apply(remove_suffixes)

# Display the updated DataFrame
df.head()

,@partOfSpeech,@lexfile,members
0,a,adj.ppl,avenged
1,a,adj.ppl,unavenged
2,a,adj.ppl,beaten
3,a,adj.ppl,calibrated
3,a,adj.ppl,graduated


In [ ]:
df['members'].apply(lambda x: " ".join(x.split("_")))

0                      avenged
1                    unavenged
2                       beaten
3                   calibrated
3                    graduated
                  ...         
120130         bromoil process
120131           interfixation
120132      consonant mutation
120133                cohesion
120134    grammatical cohesion
Name: members, Length: 212071, dtype: object

In [ ]:
df.head()

,@partOfSpeech,@lexfile,members
0,a,adj.ppl,avenged
1,a,adj.ppl,unavenged
2,a,adj.ppl,beaten
3,a,adj.ppl,calibrated
3,a,adj.ppl,graduated


In [ ]:
pd.get_dummies(df['@partOfSpeech'])

,a,n,r,s,v
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
3,1,0,0,0,0
...,...,...,...,...,...
120130,0,1,0,0,0
120131,0,1,0,0,0
120132,0,1,0,0,0
120133,0,1,0,0,0


In [ ]:
pd.get_dummies(df['@lexfile'])

,adj.all,adj.pert,adj.ppl,adv.all,noun.Tops,noun.act,noun.animal,noun.artifact,noun.attribute,noun.body,...,verb.consumption,verb.contact,verb.creation,verb.emotion,verb.motion,verb.perception,verb.possession,verb.social,verb.stative,verb.weather
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_pos = pd.concat([df['members'], pd.get_dummies(df['@partOfSpeech'])], axis=1)

In [ ]:
df_pos.head()

,members,a,n,r,s,v
0,avenged,1,0,0,0,0
1,unavenged,1,0,0,0,0
2,beaten,1,0,0,0,0
3,calibrated,1,0,0,0,0
3,graduated,1,0,0,0,0


In [ ]:
df_pos.shape

(212071, 6)

In [ ]:
grouped = df_pos.groupby('members').max()

In [ ]:
grouped.shape

(153361, 5)

In [ ]:
grouped.reset_index(inplace=True)
grouped.head(40)

,index,members,a,n,r,s,v
0,0,.22,0,1,0,0,0
1,1,.22-caliber,1,0,0,0,0
2,2,.22-calibre,1,0,0,0,0
3,3,.22_caliber,1,0,0,0,0
4,4,.22_calibre,1,0,0,0,0
5,5,.38-caliber,1,0,0,0,0
6,6,.38-calibre,1,0,0,0,0
7,7,.38_caliber,1,0,0,0,0
8,8,.38_calibre,1,0,0,0,0
9,9,.45-caliber,1,0,0,0,0


Copy pastes helper functions from notebooks/Pythia-12B Embedding Analysis.ipynb

In [ ]:
!git lfs clone https://huggingface.co/jstephencorey/pythia-12b-embeddings.git ./embeds/

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into './embeds'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (10/10), 3.26 KiB | 1.09 MiB/s, done.


In [ ]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
import matplotlib.pyplot as plt
import numpy as np
embedding_filename = "./embeds/pythia-12b.pth"
embedding_layer = torch.load(embedding_filename, map_location=torch.device(device))
embedding_weights = embedding_layer.weight.data.cpu().numpy()
print(embedding_weights)

cpu
[[ 0.00230217 -0.00296211  0.00490189 ... -0.00183678  0.00023139
  -0.00206184]
 [ 0.01398468 -0.00607681  0.02134705 ... -0.00453568 -0.0137558
  -0.00044656]
 [ 0.00622177 -0.01074982  0.00720215 ...  0.00630188  0.00068235
  -0.02189636]
 ...
 [-0.01255035 -0.00182629 -0.0049057  ... -0.00560379  0.00989532
   0.01010895]
 [ 0.00396347  0.00630188 -0.01152802 ... -0.00132656 -0.01219177
   0.00511551]
 [ 0.00543213 -0.00561523 -0.01293182 ...  0.00473022 -0.00455093
  -0.0066452 ]]


# Preprocessing
I am pretty sure we want to do some slightly more involved matching of wordnet words to tokens here. This needs to account for: variations in whitespace, capitalization, maybe how apostrophes are represented, I am sure at least one more thing I have forgotten.

I have however burned out on brain cells for the day so I'm passing this off.

HERE ENDETH THE CODE THAT IS PROPERLY WORKED OVER

In [ ]:
from transformers import AutoTokenizer
model_name = "EleutherAI/pythia-1.3b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# split dataset into X, y for sklearn
_X, _y = list(dataset.keys()), list(dataset.values())
# tokenize the X
_X = tokenizer(_X, add_special_tokens=False, padding=False)['input_ids']
# pass tokenized X through embedding layers
X, y = [], []
for i, x in enumerate(_X):
    # filter our 1 token terms
    if len(x) == 1:
        token = embedding_layer(torch.tensor(x, device=device)).detach().cpu().squeeze().numpy()
        X.append(token)
        y.append(_y[i])

X[:3], y[:3], len(X), len(y)

([array([-0.01623535,  0.0160675 , -0.01928711, ...,  0.01625061,
          0.00311661,  0.04000854], dtype=float32),
  array([ 0.02862549, -0.03579712,  0.01263428, ..., -0.00881195,
         -0.01029968, -0.00348091], dtype=float32),
  array([ 0.00106907,  0.00740051,  0.02050781, ..., -0.0174408 ,
          0.00797272,  0.03890991], dtype=float32)],
 [[1, 0], [1, 1], [1, 1]],
 3766,
 3766)

We don't have many samples to work with. Oh well, train test split now.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1337)
len(X_train), len(X_test), len(y_train), len(y_test)

(2824, 942, 2824, 942)

## multioutput classifier
per sklearn docs
> This strategy consists of fitting one classifier per target. This is a simple strategy for extending classifiers that do not natively support multi-target classification.

so i'm going to *yoink* the example in the docs

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

clf = MultiOutputClassifier(LogisticRegression(max_iter=100000)).fit(X_train, y_train)
clf

MultiOutputClassifier(estimator=LogisticRegression(max_iter=100000))

Check the proportion of labels that we have.

In [ ]:
np.unique(y_test, return_counts=True, axis=0)

(array([[0, 1],
        [1, 0],
        [1, 1]]),
 array([ 46, 616, 280]))

In [ ]:
from sklearn.metrics import accuracy_score, hamming_loss

y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred), hamming_loss(y_test, y_pred)

(0.8131634819532909, 0.10031847133757962)

Hey that's not bad! See for hamming loss: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.hamming_loss.html

Given that the hamming loss + zero-one accuracy score adds up to about 1, we can conclude that the terms that were incorrectly classified, weren't the ones that were both verbs and nouns (since hamming loss is upper bounded by zero-one loss, and its pretty much hitting the zero-one loss)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_test, y_pred)

array([[[  1,  45],
        [  0, 896]],

       [[576,  40],
        [104, 222]]])

I'm not qualified to interpret this, so I'm just going to make a PR and push this :)